## Bell State Preparation from Cirq

In [1]:
import cirq
from jaqalpaq.transpilers.cirq import jaqal_circuit_from_cirq_circuit
from jaqalpaq.generator import generate_jaqal_program

In [2]:
cirq_bell = cirq.Circuit()
qubits = [cirq.LineQubit(0), cirq.LineQubit(1)]
cirq_bell.append(cirq.H.on(qubits[0]))
cirq_bell.append(cirq.CNOT(*qubits))
print(cirq_bell)

0: ───H───@───
          │
1: ───────X───


In [3]:
cirq_ion_bell = cirq.ConvertToIonGates().convert_circuit(cirq_bell)
print(cirq_ion_bell)

0: ───PhX(1)───────────MS(0.25π)───PhX(-0.5)^0.5───S^-1───
                       │
1: ────────────────────MS(0.25π)───PhX(1)^0.5─────────────


In [4]:
jaqal_bell = jaqal_circuit_from_cirq_circuit(cirq_ion_bell)
print(generate_jaqal_program(jaqal_bell))

register allqubits[2]

prepare_all
R allqubits[0] 3.141592653589793 3.141592653589793
MS allqubits[0] allqubits[1] 0 1.5707963267948966
<
	R allqubits[0] -1.5707963267948972 1.5707963267948966
	R allqubits[1] 3.141592653589793 1.5707963267948966
>
Rz allqubits[0] -1.5707963267948966
measure_all



## Bell State Preparation from Quil

In [5]:
import pyquil
from pyquil.gates import *
from numpy import pi
from jaqalpaq.transpilers.quil import get_ion_qc, quil_gates

In [6]:
qg = quil_gates()
MS = qg["MS"]
R = qg["R"]
quil_bell = pyquil.Program()
quil_bell += X(0)
quil_bell += MS(0, pi/2, 0, 1)
quil_bell += R(-pi/2, pi/2, 0)
quil_bell += R(pi, pi/2, 1)
quil_bell += RZ(-pi/2, 0)
print(quil_bell)

X 0
MS(0,pi/2) 0 1
R(-pi/2,pi/2) 0
R(pi,pi/2) 1
RZ(-pi/2) 0



In [7]:
quil_qc = get_ion_qc(2)
jaqal_bell = quil_qc.compile(quil_bell)
print(generate_jaqal_program(jaqal_bell))

register qreg[2]

{
	prepare_all
	Px qreg[0]
	MS qreg[0] qreg[1] 0.0 1.5707963267948966
	R qreg[0] -1.5707963267948966 1.5707963267948966
	R qreg[1] 3.141592653589793 1.5707963267948966
	Rz qreg[0] -1.5707963267948966
	measure_all
}



## Bell State Preparation from ProjectQ

In [8]:
import projectq
from projectq.cengines import MainEngine, DummyEngine
from projectq.ops import H, CNOT, Measure, All
from jaqalpaq.transpilers.projectq import get_engine_list, JaqalBackend

In [9]:
backend = JaqalBackend()
engine_list = get_engine_list()
engine = MainEngine(backend, engine_list, verbose=True)
q1 = engine.allocate_qubit()
q2 = engine.allocate_qubit()
H | q1
CNOT | (q1, q2)
All(Measure) | [q1, q2]
engine.flush()
print(generate_jaqal_program(backend.circuit))

register q[2]

{
	prepare_all
	R q[0] 0 3.14159265359
	R q[0] 1.5707963267948966 3.141592653589413
	R q[0] 0 4.712388980384414
	R q[1] 0 1.570796326795
	MS q[0] q[1] 0 1.570796326795
	R q[0] 1.5707963267948966 1.570796326795
	measure_all
}



## The Automated Scheduler

In [10]:
from jaqalpaq.scheduler import schedule_circuit

In [11]:
scheduled_circuit = schedule_circuit(backend.circuit)
print(generate_jaqal_program(scheduled_circuit))

register q[2]

{
	prepare_all
	<
		R q[0] 0 3.14159265359
		R q[1] 0 1.570796326795
	>
	R q[0] 1.5707963267948966 3.141592653589413
	R q[0] 0 4.712388980384414
	MS q[0] q[1] 0 1.570796326795
	R q[0] 1.5707963267948966 1.570796326795
	measure_all
}



## Detailed Example: VQE Transpilation from Qiskit

### Constructing the VQE Circuit in Qiskit

In [12]:
import qiskit
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.circuit.library.ansatzes import UCCSD
from qiskit_nature.circuit.library.initial_states import HartreeFock

from qiskit.transpiler import PassManager

from jaqalpaq.transpilers.qiskit import jaqal_circuit_from_qiskit_circuit, ion_pass_manager, get_ion_instance
from jaqalpaq.run import run_jaqal_circuit

In [13]:
molecule = Molecule(
    geometry=[["H", [0.0, 0.0, 0.0]], ["H", [0.735, 0.0, 0.0]]], charge=0, multiplicity=1
)
driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto6g", driver_type=ElectronicStructureDriverType.PYSCF
)

es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()

qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
qubit_op = qubit_converter.convert(second_q_op[0], num_particles=es_problem.num_particles)

num_spin_orbitals = 4
initial_state = HartreeFock(num_spin_orbitals, es_problem.num_particles, qubit_converter)
uccsd = UCCSD()
uccsd.qubit_converter = qubit_converter
uccsd.num_particles = es_problem.num_particles
uccsd.initial_state = initial_state
uccsd.num_spin_orbitals = num_spin_orbitals

params = (4.881817576986406e-05, -1.5180312771430236e-05, -0.07605047960428524)
superconducting_circuit = uccsd.assign_parameters(dict(zip(uccsd.parameters, params))).decompose()
print(superconducting_circuit)

     ┌───┐┌───────────────────────────────────┐»
q_0: ┤ X ├┤0                                  ├»
     └───┘│  exp(-it IY)(4.88181757698641e-5) │»
q_1: ─────┤1                                  ├»
          └───────────────────────────────────┘»
«     ┌────────────────────────────────────┐»
«q_0: ┤0                                   ├»
«     │  exp(-it YI)(-1.51803127714302e-5) │»
«q_1: ┤1                                   ├»
«     └────────────────────────────────────┘»
«     ┌──────────────────────────────────────────┐
«q_0: ┤0                                         ├
«     │  exp(-it (XY + YX))(-0.0760504796042852) │
«q_1: ┤1                                         ├
«     └──────────────────────────────────────────┘


### Unrolling to Ion Gates

In [14]:
ion_circuit = ion_pass_manager().run(superconducting_circuit)
print(ion_circuit)

                    ┌───┐               ┌─────────────────────────────────┐»
q_0: ───────────────┤ X ├───────────────┤ Jaqalr(π/2,9.76363515397281e-5) ├»
     ┌──────────────┴───┴──────────────┐└──────────────┬───┬──────────────┘»
q_1: ┤ Jaqalr(π/2,3.03606255428605e-5) ├───────────────┤ Z ├───────────────»
     └─────────────────────────────────┘               └───┘               »
«     ┌─────┐┌───┐ ┌────┐┌─────────────────┐┌──────┐┌─────────────────────────┐»
«q_0: ┤ Sdg ├┤ Z ├─┤ Sy ├┤1                ├┤ √Xdg ├┤ Rz(-0.0760504796042852) ├»
«     └┬────┤├───┴┐└────┘│  Jaqalms(0,π/2) │├──────┤└─────────┬──────┬────────┘»
«q_1: ─┤ Sy ├┤ Sy ├──────┤0                ├┤ √Xdg ├──────────┤ Sydg ├─────────»
«      └────┘└────┘      └─────────────────┘└──────┘          └──────┘         »
«           ┌─────────────────┐┌──────┐ ┌───┐  ┌────┐┌───┐  ┌───┐ ┌────┐      »
«q_0: ──────┤1                ├┤ √Xdg ├─┤ Z ├──┤ Sy ├┤ S ├──┤ Z ├─┤ Sy ├──────»
«     ┌────┐│  Jaqalms(0,π/2) │├──────┤┌┴───┴─┐├──

### Transpiling to Jaqal

In [15]:
jaqal_circuit = jaqal_circuit_from_qiskit_circuit(ion_circuit)
print(generate_jaqal_program(jaqal_circuit))

register baseregister[2]

map q baseregister[0:2:1]

{
	prepare_all
	Px q[0]
	R q[0] 1.5707963267948966 9.76363515397281e-05
	Szd q[0]
	Pz q[0]
	Sy q[0]
	R q[1] 1.5707963267948966 3.0360625542860465e-05
	Pz q[1]
	Sy q[1]
	Sy q[1]
	MS q[1] q[0] 0.0 1.5707963267948966
	Sxd q[0]
	Rz q[0] -0.07605047960428522
	Sxd q[1]
	Syd q[1]
	Sy q[1]
	MS q[1] q[0] 0.0 1.5707963267948966
	Sxd q[0]
	Pz q[0]
	Sy q[0]
	Sz q[0]
	Pz q[0]
	Sy q[0]
	Sxd q[1]
	Syd q[1]
	Pz q[1]
	Sy q[1]
	Szd q[1]
	Pz q[1]
	Sy q[1]
	Sy q[1]
	MS q[1] q[0] 0.0 1.5707963267948966
	Sxd q[0]
	Rz q[0] 0.07605047960428522
	Sxd q[1]
	Syd q[1]
	Sy q[1]
	MS q[1] q[0] 0.0 1.5707963267948966
	Sxd q[0]
	Pz q[0]
	Sy q[0]
	Sxd q[1]
	Syd q[1]
	Pz q[1]
	Sy q[1]
	Sz q[1]
	measure_all
}



### Scheduling and Emulation

In [16]:
scheduled_jaqal_circuit = schedule_circuit(jaqal_circuit)
print(generate_jaqal_program(scheduled_jaqal_circuit))

register baseregister[2]

map q baseregister[0:2:1]

{
	prepare_all
	<
		Px q[0]
		R q[1] 1.5707963267948966 3.0360625542860465e-05
	>
	<
		R q[0] 1.5707963267948966 9.76363515397281e-05
		Pz q[1]
	>
	<
		Szd q[0]
		Sy q[1]
	>
	<
		Pz q[0]
		Sy q[1]
	>
	Sy q[0]
	MS q[1] q[0] 0.0 1.5707963267948966
	<
		Sxd q[0]
		Sxd q[1]
	>
	<
		Rz q[0] -0.07605047960428522
		Syd q[1]
	>
	Sy q[1]
	MS q[1] q[0] 0.0 1.5707963267948966
	<
		Sxd q[0]
		Sxd q[1]
	>
	<
		Pz q[0]
		Syd q[1]
	>
	<
		Sy q[0]
		Pz q[1]
	>
	<
		Sz q[0]
		Sy q[1]
	>
	<
		Pz q[0]
		Szd q[1]
	>
	<
		Sy q[0]
		Pz q[1]
	>
	Sy q[1]
	Sy q[1]
	MS q[1] q[0] 0.0 1.5707963267948966
	<
		Sxd q[0]
		Sxd q[1]
	>
	<
		Rz q[0] 0.07605047960428522
		Syd q[1]
	>
	Sy q[1]
	MS q[1] q[0] 0.0 1.5707963267948966
	<
		Sxd q[0]
		Sxd q[1]
	>
	<
		Pz q[0]
		Syd q[1]
	>
	<
		Sy q[0]
		Pz q[1]
	>
	Sy q[1]
	Sz q[1]
	measure_all
}



In [17]:
run_jaqal_circuit(scheduled_jaqal_circuit).subcircuits[0].probability_by_str

OrderedDict([('00', 2.383214283044736e-09),
             ('10', 0.9942274637711583),
             ('01', 0.005772533615185433),
             ('11', 2.3044189527351244e-10)])

## Optimizing Circuits for Ion Hardware with t|ket>

In [18]:
from jaqalpaq.transpilers.tket import jaqal_circuit_from_tket_circuit
from pytket.predicates import CompilationUnit
from pytket.extensions.qiskit import qiskit_to_tk
from pytket.passes import SynthesiseUMD, DecomposeBoxes

In [19]:
tket_circuit = qiskit_to_tk(superconducting_circuit)
tket_circuit

[X q[0]; CircBox q[0], q[1]; CircBox q[0], q[1]; CircBox q[0], q[1]; ]

In [20]:
unit = CompilationUnit(tket_circuit)
DecomposeBoxes().apply(unit)
SynthesiseUMD().apply(unit)
tket_jaqal_circuit = jaqal_circuit_from_tket_circuit(unit.circuit)
print(generate_jaqal_program(tket_jaqal_circuit))

register baseregister[2]

map q baseregister[0:2:1]

{
	prepare_all
	Rz q[0] 10.995574287564276
	Rz q[1] 4.71238898038469
	R q[0] 1.5707963267948966 1.5707963267948966
	MS q[1] q[0] 0 1.5707963267948966
	Rz q[0] 1.5707963267948966
	R q[1] 0.0 4.71238898038469
	R q[0] 3.141592653589793 7.853981633974483
	MS q[1] q[0] 0 1.5707963267948966
	Rz q[0] 9.42477796076938
	R q[1] 1.5707963267948966 1.5707963267948966
	R q[0] 1.5707963267948966 1.5707963267948966
	MS q[1] q[0] 0 1.5707963267948966
	R q[0] 0.7853981633974483 3.141592653589793
	R q[1] 0.0 4.71238898038469
	MS q[1] q[0] 0 1.5707963267948966
	measure_all
}



In [21]:
optimized_jaqal_circuit = schedule_circuit(tket_jaqal_circuit)
print(generate_jaqal_program(optimized_jaqal_circuit))

register baseregister[2]

map q baseregister[0:2:1]

{
	prepare_all
	<
		Rz q[0] 10.995574287564276
		Rz q[1] 4.71238898038469
	>
	R q[0] 1.5707963267948966 1.5707963267948966
	MS q[1] q[0] 0 1.5707963267948966
	<
		Rz q[0] 1.5707963267948966
		R q[1] 0.0 4.71238898038469
	>
	R q[0] 3.141592653589793 7.853981633974483
	MS q[1] q[0] 0 1.5707963267948966
	<
		Rz q[0] 9.42477796076938
		R q[1] 1.5707963267948966 1.5707963267948966
	>
	R q[0] 1.5707963267948966 1.5707963267948966
	MS q[1] q[0] 0 1.5707963267948966
	<
		R q[0] 0.7853981633974483 3.141592653589793
		R q[1] 0.0 4.71238898038469
	>
	MS q[1] q[0] 0 1.5707963267948966
	measure_all
}



In [22]:
run_jaqal_circuit(optimized_jaqal_circuit).subcircuits[0].probability_by_str

OrderedDict([('00', 7.703719777548946e-33),
             ('10', 1.0),
             ('01', 2.0029671421627253e-31),
             ('11', 1.248002603962929e-31)])